In [1]:
#%load_ext tikzmagic
from IPython.display import display, clear_output
import ipywidgets as widgets
from imrestore import shell
import cv2
from IPython.display import HTML
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  } 
  
  $( document ).ready(function(){
    code_shown=false; 
    $('div.input').hide()
  });
</script>
''')
# <form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

# Welcome to ImRestore program demo

In [2]:
all_widgets = []
all_helpers = []

# header
#all_widgets.append(widgets.Label("Welcome to ImRestore program"))

# filenames
filenames = widgets.Text(
    value='',
    placeholder='Type path to images',
    description='Filenames:',
    disabled=False
)
all_widgets.append(filenames)
all_helpers.append(("store",[""],lambda:filenames.value))

# v = "verbosity",
verbosity = widgets.Dropdown(
    options={'Do not print messages':0, 'Normal messages':1, 
             'Normal and debug messages':2, 'All messages and show main results':3,
             'All messages and show all results':4,'All messages, all results and additional data':5},
    value=1,
    description='Verbosity:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)
all_widgets.append(verbosity)
all_helpers.append(("store",["-v","--verbosity"],lambda:verbosity.value))

# f="feature",
feature_ = widgets.Label(
    value="Feature detector: ",
    placeholder='Feature detector',
    description='Configure detector and matcher',
    disabled=False
)

feature_a = widgets.Checkbox(
    value=True,
    description='ASIFT',
    disabled=False
)

feature_b = widgets.Dropdown(
    options={'sift':'sift', 'surf':'surf', 'orb':'orb', 'brisk':'brisk'},
    value='sift',
    description='Detector:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)

feature_c = widgets.Checkbox(
    value=True,
    description='FLANN', # Fast Approximate Nearest Neighbor Search Library
    disabled=False
)

all_widgets.append(widgets.HBox([feature_,feature_a,feature_b,feature_c]))
all_helpers.append(("store",["-f","--feature"],lambda:"-".join([str(i) for i in (("","a")[feature_a.value],feature_b.value,("","flann")[feature_c.value]) if i])))

# u="pool",
NO_CPU = cv2.getNumberOfCPUs()
options = [None]
options.extend([i+1 for i in range(NO_CPU)])
pool = widgets.SelectionSlider(
    options=options,
    value=None,
    description='No. of CPUs: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
#     readout_format='i',
#     slider_color='black'
)
all_widgets.append(pool)
all_helpers.append(("store",["-u","--pool"],lambda:pool.value))

# c="cachePath",
cachePath = widgets.Text(
    value='',
    placeholder='Type path to cache date from program',
    description='Cache path:',
    disabled=False
)
all_widgets.append(widgets.HBox([cachePath,widgets.Label('Ex: "." places the path in the current directory')]))
all_helpers.append(("store",["-c","--cachePath"],lambda:cachePath.value))

# e="clearCache",
clearCache = widgets.Dropdown(
    options={'Do not clear':0, 'Check data integrity of previous session before use':1,
             'Re-compute data but other cache data is left intact':2, 'All CachePath is cleared before use':3},
    value=0,
    description='Cache Mode:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)
all_widgets.append(clearCache)
all_helpers.append(("store",["-e","--clearCache"],lambda:clearCache.value))
    
# loader="loader",
loader = widgets.Dropdown(
    options={'BGRA':-1, 'Gray':0, 'BGR':1},
    value=1,
    description='Color:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)
all_widgets.append(loader)
all_helpers.append(("store",["--loader"],lambda:"{},".format(loader.value)))

# p="process_shape",
ps_width = widgets.IntText(
    value=0,
    description='Width:',
    disabled=False
)
ps_height = widgets.IntText(
    value=0,
    description='Height:',
    disabled=False
)
all_widgets.append(widgets.HBox([widgets.Label('Process shape'),ps_width,ps_height]))
all_helpers.append(("store",["-p","--process_shape"],lambda:",".join([(str(i.value) if i.value != 0 else "None") 
                                                                      for i in (ps_width,ps_height) if (ps_width.value or ps_height.value)])))

# l="load_shape",
ls_width = widgets.IntText(
    value=0,
    description='Width:',
    disabled=False
)
ls_height = widgets.IntText(
    value=0,
    description='Height:',
    disabled=False
)
all_widgets.append(widgets.HBox([widgets.Label('Load shape'),ls_width,ls_height]))

all_helpers.append(("store",["-l","--load_shape"],lambda:",".join([(str(i.value) if i.value != 0 else "None") 
                                                                      for i in (ls_width,ls_height) if (ls_width.value or ls_height.value)])))

# b="baseImage",
baseImage = widgets.Text(
    value='',
    placeholder='Type base Image or leave empty to select best image',
    description='Base Image:',
    disabled=False
)
all_widgets.append(baseImage)
all_helpers.append(("store",["-b","--baseImage"],lambda:baseImage.value))

# m="selectMethod",
selectMethod = widgets.Dropdown(
    options={'Best matches':None, 'Correlation':'Correlation', 'Chi-squared':'Chi-squared', 
             'Intersection':'Intersection','Hellinger':'Hellinger','Entropy':'Entropy'},
    value=None,
    description='Mergin order:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)
all_widgets.append(selectMethod)
all_helpers.append(("store",["-m","--selectMethod"],lambda:selectMethod.value))

# d="distanceThresh",
distanceThresh = widgets.FloatSlider(
    value=0.75,
    min=0.0,
    max=1.0,
    step=0.01,
    description='distanceThresh:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    slider_color='white'
)
all_widgets.append(distanceThresh)
all_helpers.append(("store",["-d","--distanceThresh"],lambda:distanceThresh.value))

# i="inlineThresh",
inlineThresh = widgets.FloatSlider(
    value=0.2,
    min=0.0,
    max=1.0,
    step=0.01,
    description='inlineThresh:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    slider_color='white'
)
all_widgets.append(inlineThresh)
all_helpers.append(("store",["-i","--inlineThresh"],lambda:inlineThresh.value))

# r="rectangularityThresh",
rectangularityThresh = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description='rectangularityThresh:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    slider_color='white'
)
all_widgets.append(rectangularityThresh)
all_helpers.append(("store",["-r","--rectangularityThresh"],lambda:rectangularityThresh.value))

# j="ransacReprojThreshold",
ransacReprojThreshold = widgets.FloatSlider(
    value=10.0,
    min=0.0,
    max=100.0,
    step=0.1,
    description='ransacReprojThreshold:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    slider_color='white'
)
all_widgets.append(ransacReprojThreshold)
all_helpers.append(("store",["-j","--ransacReprojThreshold"],lambda:ransacReprojThreshold.value))


# n="centric",
centric = widgets.Checkbox(
    value=False,
    description='Centric',
    disabled=False
)
all_widgets.append(centric)
all_helpers.append(("store_true",["-n","--centric"],lambda:centric.value))

# t="hist_match",
hist_match = widgets.Checkbox(
    value=False,
    description='Histogram matching',
    disabled=False
)
all_widgets.append(hist_match)
all_helpers.append(("store_true",["-t","--hist_match"],lambda:hist_match.value))

# s="save",
# o="overwrite",
overwrite = widgets.Checkbox(
    value=False,
    description='Overwrite result',
    disabled=False
)
all_widgets.append(overwrite)
all_helpers.append(("store_true",["-o","--overwrite"],lambda:overwrite.value))

# g="grow_scene",
grow_scene = widgets.Checkbox(
    value=False,
    description='Grow scene',
    disabled=False
)
all_widgets.append(grow_scene)
all_helpers.append(("store_true",["-g","--grow_scene"],lambda:grow_scene.value))

# y="denoise",
denoise = widgets.Dropdown(
    options=[None,"mild", "normal", "heavy"],
    value=None,
    description='Denoise:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)
all_widgets.append(denoise)
all_helpers.append(("store",["-y","--denoise"],lambda:denoise.value))

# a="lens",
lens = widgets.Checkbox(
    value=False,
    description='Lens simulation',
    disabled=False
)
all_widgets.append(lens)
all_helpers.append(("store_true",["-a","--lens"],lambda:lens.value))

# k="enclose",
enclose = widgets.Checkbox(
    value=False,
    description='Enclose object',
    disabled=False
)
all_widgets.append(enclose)
all_helpers.append(("store_true",["-k","--enclose"],lambda:enclose.value))

# z="restorer",
restorer = widgets.Dropdown(
    options=['RetinalRestore', 'ImRestore'],
    value='RetinalRestore',
    description='Restorer:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)
all_widgets.append(restorer)
all_helpers.append(("store",["-z","--restorer"],lambda:restorer.value))

# x="expert",
expert = widgets.Text(
    value='',
    placeholder='Type path to expert data',
    description='Expert path:',
    disabled=False
)
all_widgets.append(expert)
all_helpers.append(("store",["-x","--expert"],lambda:expert.value))

# q="console",
# w="debug",
# onlykeys="onlykeys"
onlykeys = widgets.Checkbox(
    value=False,
    description='Only Keys',
    disabled=False
)
all_widgets.append(onlykeys)
all_helpers.append(("store_true",["--onlykeys"],lambda:onlykeys.value))

### to run program
        
def find_short_parameter(flags, short = False):
    flag = ""
    tries = (0,1,2)
    c = [i.count("-") for i in flags] # find number of 
    for t in tries:
        try:
            ind = c.index(t)
            flag = flags[ind]
            if t==1 and short:
                break # breaks when short paramenter
        except ValueError:
            pass
    return flag

def get_command(helpers,short = False):
    commands = [] # init command flags
    for action, flags, helper in all_helpers:
        #print(flags) debugg flag
        if action=="store":
            cmd = helper()
            if cmd:
                commands.append("{} {}".format(find_short_parameter(flags,short),cmd))
        elif action=="store_true":
            if helper():
                commands.append("{}".format(find_short_parameter(flags,short)))
        elif action=="store_false":
            if not helper():
                commands.append("{}".format(find_short_parameter(flags,short)))
        else:
            raise Exception("Action '{}' in flags '{}' not recognized".format(action,))

    return (" ".join(commands))


# run button
run = widgets.Button(
    value=False,
    description='Run!',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run program with generated command',
    icon=''
)
# see coloring in http://stackoverflow.com/a/16816874/5288758
def run_program(button=None):
    command = change_command_label()
    button.description = "Running..."
    print("\x1b[32m{}\x1b[0m".format("Running ImRestore command: '{}'".format(command)))
    try:
        res = shell(command.split())
        # print profiling
        print(res.restorer.profiler.string_structured(True))
        if not onlykeys.value:
            # print restored image
            from pylab import figure, imshow, show
            figure()
            imshow(cv2.cvtColor(res.restorer.restored,cv2.COLOR_BGR2RGB))
            show()
    except Exception as e:
        #raise
        print("\x1b[31m{}\x1b[0m".format(e))
    button.description = "Run!"
    
run.on_click(run_program)

# clear output
clear = widgets.Button(
    value=False,
    description="Clear output",
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clear log output',
    icon='check'
)
clear.on_click(lambda x: clear_output())
all_widgets.append(widgets.HBox([run,clear]))

# command
def change_command_label(label=None):
    global command_label
    command = get_command(all_helpers,command_short.value)
    command_label.value = command
    return command

command_short = widgets.Checkbox(
    value=False,
    description='Short Notation',
    disabled=False
)

command_label = widgets.Label(
    value=get_command(all_helpers,command_short.value),
    placeholder='Command...',
    description='Command...',
    disabled=False
)
all_widgets.append(widgets.HBox([widgets.Label("Command:"),command_label,command_short]))

# bind widgets with command generator
def bind_to_widgets(widget_list,event,exclude = ()):
    """bind an event function to be observed in by widgets"""
    for widget in widget_list:
        if isinstance(widget,widgets.Box):
            bind_to_widgets(widget.children,event,exclude)
        elif not exclude or not isinstance(widget,exclude):
            #print(widget.value,type(widget.value)) # used for debugging types
            widget.observe(event)
            
# observe all widgets except buttons
bind_to_widgets(all_widgets,change_command_label, (widgets.Button,))

# organize widgets
widgets.VBox(all_widgets)